In [1]:
import wandb
from datetime import datetime

In [2]:
batch_size = 16
lr = 1e-3
wd = 1e-5

In [3]:
wandb_entity='longyi'
model_name = "resnet50"
wandb.init(project="cervical-spine", entity=wandb_entity, config={
    "model":model_name,
    "batch_size":batch_size,
    "lr" : lr,
    "wd" : wd
})
wandb.run.name = f'xray_{model_name}_' + datetime.now().strftime("%H%M%S")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: longyi. Use `wandb login --relogin` to force relogin


In [4]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from tqdm import tqdm

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from utils.dcm_utils import *
from utils.nii_utils import *
from utils.train_utils import *

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.io import read_image
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, CenterCrop
import torchvision.transforms.functional as TF

In [6]:
DATA_DIR = "/media/longyi/SSD9701/"
TRAIN_XRAY_DIR = os.path.join(DATA_DIR, "xray_images")
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")

In [7]:
class XrayDataset(Dataset):
    def __init__(self, xray_dir, label_df, transform=None, target_transform=None):
        self.xray_dir = xray_dir
        self.label_df = label_df

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.label_df)

    def __getitem__(self, idx):
        row = self.label_df.iloc[idx]
        UID = row.name
        label = torch.tensor(row[['patient_overall', 'C1','C2','C3','C4','C5','C6','C7']])

        axial = read_image(os.path.join(self.xray_dir, UID, 'axial.jpeg')).float()
        sagittal = read_image(os.path.join(self.xray_dir, UID, 'sagittal.jpeg')).float()
        coronal = read_image(os.path.join(self.xray_dir, UID, 'coronal.jpeg')).float()

        if self.transform:
            axial = self.transform(axial)
            sagittal = self.transform(sagittal)
            coronal = self.transform(coronal)

        if self.target_transform:
            label = self.target_transform(label)
        return (axial, sagittal, coronal), label

In [8]:
df = pd.read_csv(os.path.join(DATA_DIR, 'train_clean.csv')).set_index('StudyInstanceUID')
len(df)

2012

In [9]:
df.head()

,patient_overall,C1,C2,C3,C4,C5,C6,C7
StudyInstanceUID,,,,,,,,
1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


In [10]:
total_len = len(df)
train_to = int(total_len * 0.8)
train_df = df.iloc[:train_to]
eval_df = df.iloc[train_to:]

print(f"train {len(train_df)} eval {len(eval_df)}")

train 1609 eval 403


In [11]:
class ImageTransform:
    """Rotate by one of the given angles."""

    def __init__(self, wh):
        self.wh = wh

    def __call__(self, x):
        h, w = float(x.shape[1]), float(x.shape[2])

        if h > w:
            x = TF.resize(x, [int(self.wh), int(self.wh * w / h)])
        else:
            x = TF.resize(x, [int(self.wh * h / w), int(self.wh)])

        x = TF.center_crop(x, self.wh)

        return x

In [12]:
transform = Compose([
    Normalize(255 * 0.5, 255 * 0.5),
    ImageTransform(224.)
])
target_transform = None
train_dataset = XrayDataset(TRAIN_XRAY_DIR, train_df, transform=transform, target_transform=target_transform)
eval_dataset = XrayDataset(TRAIN_XRAY_DIR, eval_df, transform=transform, target_transform=target_transform)
train_dataset[0]

((tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]]),
  tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]),
  tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]])),
 tensor([1, 1, 1, 0, 0, 0, 0, 0]))

In [13]:
(a, s, c), y = train_dataset[0]
print(f"{s.min()} {s.max()}")

-1.0 0.8939855098724365


In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=6)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=6)

## Model

In [15]:
class XrayModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.sagittal_feature = self._make_feature_extractor()
        self.coronal_feature = self._make_feature_extractor()
        self.fc = nn.Sequential(
            nn.Conv2d(2048 * 2, 1, kernel_size=1),
            nn.AdaptiveAvgPool2d((7, 1)),
            nn.Flatten()
        )

    def _make_feature_extractor(self):
        feature = torchvision.models.resnet50(pretrained=True)
        conv1_weight = feature.conv1.weight
        new_conv1_weight = conv1_weight.mean(dim=1).unsqueeze(1)

        feature.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=2, padding=3, bias=False)
        feature.conv1.weight = nn.Parameter(new_conv1_weight, requires_grad=True)
        feature.avgpool = nn.AdaptiveAvgPool2d((7, 1))

        return nn.Sequential(
            *list(feature.children())[:-1]
        )

    def forward(self, x):
        (_, sagittal, coronal) = x
        sagittal_feature = self.sagittal_feature(sagittal)
        coronal_feature = self.coronal_feature(coronal)

        out = torch.cat((sagittal_feature, coronal_feature), dim=1)
        out = self.fc(out)

        return out

In [16]:
model = XrayModel()

In [17]:
input = torch.randn(2, 1, 224, 224)
out = model((None, input, input))
out.shape

torch.Size([2, 7])

## Train

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [19]:
model = model.to(device)

In [20]:
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

In [21]:
def loss_fn(logits, labels):
    # logits N x 7
    # labels N x 7
    weights = calculate_weights(labels)[:, 1:]
    loss = F.binary_cross_entropy_with_logits(logits, labels[:, 1:].to(torch.float), reduction='none')
    loss = weights * loss

    weights_sum = weights.sum(dim=1)
    # overall_loss = loss[:, 0] / weights_sum
    # c_loss = loss[:, 1:].sum(dim=1) / weights_sum
    return loss.sum(dim=1) / weights_sum    # N x 1
    # return overall_loss.mean(), c_loss.mean()

In [24]:
def evaluate(epoch):
    model.eval()
    eval_iter = tqdm(eval_loader)

    losses = []
    for i, (x, y) in enumerate(eval_iter):
        x = (v.to(device) for v in x)
        y = y.to(device=device)

        logits = model(x)
        c_loss = loss_fn(logits, y).mean()
        loss = c_loss

        losses.append(loss.item())

        # accuracy
        pred = logits.sigmoid().ge(0.5).int()
        correct = (y[:, 1:] == pred).float().mean(dim=0)
        # overall_acc = correct[0]
        c_acc = correct.mean()

        eval_iter.set_description(f"e {epoch} loss {loss.item():.4f} c_acc {c_acc.item():.4f}")

        if wandb.run is not None:
            wandb.log({
                # 'eval_overall_loss' : overall_loss.item(),
                'eval_c_loss' : c_loss.item(),
                # 'eval_loss': loss.item(),
                # 'eval_overall_acc' : overall_acc.item(),
                'eval_c_acc' : c_acc.item(),
                'epoch' : epoch
            })
    return np.mean(losses)

In [25]:
evaluate(0)

e 0 loss 0.8254 c_acc 0.1429: 100%|██████████| 26/26 [00:01<00:00, 13.21it/s]


0.818001960332577

In [26]:
def train_one_epoch(epoch):
    model.train()
    train_iter = tqdm(train_loader)
    losses = []
    for i, (x, y) in enumerate(train_iter):
        x = (v.to(device) for v in x)
        y = y.to(device=device)

        logits = model(x)
        c_loss = loss_fn(logits, y).mean()
        loss = c_loss

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.)
        optimizer.step()

        losses.append(loss.item())
        train_iter.set_description(f"t {epoch} loss {loss.item():.4f}")

        if wandb.run is not None:
            wandb.log({
                'train_c_loss' : c_loss.item(),
                'train_loss': loss.item(),
                'epoch': epoch
            })
    return np.mean(losses)

In [28]:
epochs = 50

for epoch in range(epochs):

    train_loss = train_one_epoch(epoch)

    with torch.no_grad():
        eval_loss = evaluate(epoch)

    print(f"epoch {epoch} train_loss {train_loss} eval_loss {eval_loss}")

    if wandb.run is not None:
        wandb.log({
            'average_train_loss' : train_loss,
            'average_eval_loss' : eval_loss,
        })

e 0 loss 0.1576 c_acc 0.9524: 100%|██████████| 26/26 [00:01<00:00, 13.31it/s]


epoch 0 train_loss 0.42431435874193024 eval_loss 0.5007594863955791


e 1 loss 0.2270 c_acc 0.9524: 100%|██████████| 26/26 [00:02<00:00, 12.99it/s]


epoch 1 train_loss 0.4207012907113179 eval_loss 0.43375348586302537


e 2 loss 0.3153 c_acc 0.9524: 100%|██████████| 26/26 [00:01<00:00, 13.23it/s]


...

epoch 32 train_loss 0.07014604333308663 eval_loss 1.371297341126662


e 33 loss 0.2815 c_acc 0.9048: 100%|██████████| 26/26 [00:01<00:00, 13.21it/s]


epoch 33 train_loss 0.06643941046872942 eval_loss 1.265922601406391


t 34 loss 0.0643:   6%|▌         | 6/101 [00:01<00:26,  3.56it/s]


KeyboardInterrupt: 